In [25]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from data import hrrr_data, nam_data
import pandas as pd
import numpy as np
from datetime import timedelta

In [27]:
import os

dirs = os.listdir("/home/aevans/ai2es/lstm/HRRR/fh_01/2018/03")
dirs

['20180330_hrrr_fh01.parquet',
 '20180318_hrrr_fh01.parquet',
 '20180303_hrrr_fh01.parquet',
 '20180317_hrrr_fh01.parquet',
 '20180310_hrrr_fh01.parquet',
 '20180302_hrrr_fh01.parquet',
 '20180311_hrrr_fh01.parquet',
 '20180322_hrrr_fh01.parquet',
 '20180306_hrrr_fh01.parquet',
 '20180308_hrrr_fh01.parquet',
 '20180328_hrrr_fh01.parquet',
 '20180319_hrrr_fh01.parquet',
 '20180307_hrrr_fh01.parquet',
 '20180304_hrrr_fh01.parquet',
 '20180316_hrrr_fh01.parquet',
 '20180301_hrrr_fh01.parquet',
 '20180309_hrrr_fh01.parquet',
 '20180321_hrrr_fh01.parquet',
 '20180312_hrrr_fh01.parquet',
 '20180326_hrrr_fh01.parquet',
 '20180323_hrrr_fh01.parquet',
 '20180325_hrrr_fh01.parquet',
 '20180320_hrrr_fh01.parquet',
 '20180314_hrrr_fh01.parquet',
 '20180329_hrrr_fh01.parquet',
 '20180331_hrrr_fh01.parquet',
 '20180315_hrrr_fh01.parquet',
 '20180327_hrrr_fh01.parquet',
 '20180305_hrrr_fh01.parquet',
 '20180324_hrrr_fh01.parquet',
 '20180313_hrrr_fh01.parquet']

In [28]:
for f in dirs:
    print(f)
    df = pd.read_parquet(f"/home/aevans/ai2es/lstm/HRRR/fh_01/2018/03/{f}")

20180330_hrrr_fh01.parquet
20180318_hrrr_fh01.parquet
20180303_hrrr_fh01.parquet
20180317_hrrr_fh01.parquet
20180310_hrrr_fh01.parquet
20180302_hrrr_fh01.parquet
20180311_hrrr_fh01.parquet
20180322_hrrr_fh01.parquet
20180306_hrrr_fh01.parquet
20180308_hrrr_fh01.parquet
20180328_hrrr_fh01.parquet
20180319_hrrr_fh01.parquet
20180307_hrrr_fh01.parquet
20180304_hrrr_fh01.parquet
20180316_hrrr_fh01.parquet
20180301_hrrr_fh01.parquet
20180309_hrrr_fh01.parquet
20180321_hrrr_fh01.parquet
20180312_hrrr_fh01.parquet
20180326_hrrr_fh01.parquet
20180323_hrrr_fh01.parquet
20180325_hrrr_fh01.parquet
20180320_hrrr_fh01.parquet
20180314_hrrr_fh01.parquet
20180329_hrrr_fh01.parquet
20180331_hrrr_fh01.parquet
20180315_hrrr_fh01.parquet
20180327_hrrr_fh01.parquet
20180305_hrrr_fh01.parquet
20180324_hrrr_fh01.parquet
20180313_hrrr_fh01.parquet


In [15]:
df1 = pd.read_parquet(
    "/home/aevans/ai2es/lstm/HRRR/fh_01/2018/02/20180217_hrrr_fh01.parquet"
)
df1

OSError: [Errno 22] Invalid argument

In [9]:
def get_more_fh(fh, station, var, times):
    hrrr_df_0 = hrrr_data.read_hrrr_data(str(fh + 2).zfill(2))
    hrrr_df_1 = hrrr_data.read_hrrr_data(str(fh + 4).zfill(2))

    hrrr_df_0 = hrrr_df_0[hrrr_df_0["station"] == station]
    hrrr_df_1 = hrrr_df_1[hrrr_df_1["station"] == station]

    hrrr_df_0 = hrrr_df_0[["valid_time", var]]
    hrrr_df_1 = hrrr_df_1[["valid_time", var]]

    # Create a DataFrame for valid times
    df = pd.DataFrame({"valid_time": times})
    df = df.merge(hrrr_df_0, on="valid_time", suffixes=(None, f"_{station}_+2"))
    df = df.merge(hrrr_df_1, on="valid_time", suffixes=(None, f"_{station}_+4"))
    df = df.rename(columns={"t2m": f"{var}_{station}_+2"})
    # df.fillna(-999, inplace=True)

    fh2 = df[f"{var}_{station}_+2"].values
    fh4 = df[f"{var}_{station}_+4"].values

    print(len(fh2))
    print(len(fh4))

    return fh2, fh4

In [10]:
def read_nam_data(fh):
    """
    Reads and concatenates parquet files containing forecast and error data for HRRR weather models
    for the years 2018 to 2022.

    Returns:
        pandas.DataFrame: of hrrr weather forecast information for each NYSM site.
    """

    years = ["2022", "2023"]
    savedir = f"/home/aevans/nwp_bias/src/machine_learning/data/nam_data/fh{fh}/"

    # create empty lists to hold dataframes for each model
    nam_fcast_and_error = []

    # loop over years and read in parquet files for each model
    for year in years:
        for month in np.arange(1, 13):
            str_month = str(month).zfill(2)
            if (
                os.path.exists(
                    f"{savedir}NAM_{year}_{str_month}_direct_compare_to_nysm_sites_mask_water.parquet"
                )
                == True
            ):
                print(
                    f"{savedir}NAM_{year}_{str_month}_direct_compare_to_nysm_sites_mask_water.parquet"
                )
                nam_fcast_and_error.append(
                    pd.read_parquet(
                        f"{savedir}NAM_{year}_{str_month}_direct_compare_to_nysm_sites_mask_water.parquet"
                    )
                )
            else:
                continue
            gc.collect()

    # concatenate dataframes for each model
    nam_fcast_and_error_df = pd.concat(nam_fcast_and_error)
    nam_fcast_and_error_df = nam_fcast_and_error_df.dropna()

    # return dataframes for each model
    return nam_fcast_and_error_df

In [11]:
fh = 6
station = "SOUT"
var = "t2m"

In [12]:
df = pd.read_parquet(
    "/home/aevans/nwp_bias/src/machine_learning/data/nam_data/fh001/NAM_2022_04_direct_compare_to_nysm_sites_mask_water.parquet"
)
df

,latitude,longitude,t2m,sh2,d2m,r2,u10,v10,u_total,u_dir,...,cin,dswrf,dlwrf,gh,station,valid_time,level_0,index,lead time,lsm
time,,,,,,,,,,,,,,,,,,,,,
2022-04-01 00:00:00,42.040359,-77.237259,10.103218,0.007865,9.483641,96.349721,1.640804,0.421012,1.836858,254.633860,...,-51.194306,0.000000,341.605801,5502.463200,ADDI,2022-04-01 01:00:00,0.0,0.0,0.0,0.0
2022-04-01 00:00:00,42.242489,-78.039581,5.369774,0.005322,3.630354,88.805841,4.411272,0.107432,4.429688,269.082379,...,-0.020703,0.000000,331.982933,5482.838469,BELM,2022-04-01 01:00:00,0.0,0.0,0.0,0.0
2022-04-01 00:00:00,42.058430,-75.951042,12.888993,0.008826,11.316517,90.612892,-0.301365,3.275204,3.298334,175.079971,...,-12.934617,0.000000,348.131055,5527.008490,BING,2022-04-01 01:00:00,0.0,0.0,0.0,0.0
2022-04-01 00:00:00,40.631763,-73.953674,14.533216,0.009694,13.456205,93.797555,-0.555687,8.000869,8.023277,175.928714,...,-0.020703,0.000000,387.901190,5599.040353,BKLN,2022-04-01 01:00:00,0.0,0.0,0.0,0.0
2022-04-01 00:00:00,41.439930,-73.576424,13.917935,0.009672,13.142769,95.666243,-0.584071,5.937920,5.969338,174.374824,...,-0.020703,0.000000,381.762907,5576.003682,BREW,2022-04-01 01:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29 06:00:00,42.896945,-77.864666,-2.382147,0.002916,-3.703741,90.955078,2.849240,0.171882,2.854419,266.547791,...,0.005566,0.000000,216.349182,5532.245605,YORK,2022-04-29 07:00:00,610627.0,1474296.0,1.0,1.0
2022-04-29 18:00:00,42.896945,-77.864666,11.714075,0.003342,-1.903876,38.704834,1.682588,-4.068793,4.402974,337.533203,...,0.004883,854.901855,254.066956,5581.283203,YORK,2022-04-29 19:00:00,610628.0,1501220.0,1.0,1.0
2022-04-30 06:00:00,42.896945,-77.864666,0.462030,0.003015,-3.267645,76.165352,1.267696,1.891173,2.276750,213.834854,...,-0.041797,0.000000,232.245041,5602.612305,YORK,2022-04-30 07:00:00,610629.0,1528144.0,1.0,1.0


In [13]:
nam_df = read_nam_data(str(fh).zfill(3))

ValueError: No objects to concatenate

In [ ]:
# hrrr_df = hrrr_data.read_hrrr_data(str(fh).zfill(2))

# # Filter NYSM data to match valid times from HRRR data
# mytimes = hrrr_df["valid_time"].tolist()
# fh2_, fh4_ = get_more_fh(fh, station, var, mytimes)

In [ ]:
# len(mytimes)

In [ ]:
a100_mae = [
    0.07,
    0.17,
]
a100_mse = [
    0.07,
    0.22,
]
a100_batch = [
    1000,
    5000,
]
a100_gpu = [8, 30]
a100_runtime = [
    timedelta(seconds=24, minutes=16, hours=0),
    timedelta(seconds=5, minutes=16, hours=0),
]

In [ ]:
gh200_mae = [0.06, 0.06]
gh200_mse = [0.06, 0.07]
gh200_batch = [1000, 10000]
gh200_gpu = [8, 64]
gh200_runtime = [
    timedelta(seconds=22, minutes=6, hours=0),
    timedelta(seconds=51, minutes=6, hours=0),
]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta


def plot_runtime_bar_chart(a100_batch, a100_run_time):
    # Convert timedelta objects to total minutes
    run_time_minutes = [rt.total_seconds() / 60 for rt in a100_run_time]

    # Create the plot
    fig, ax = plt.subplots()

    # Plot the bar chart
    ax.bar(a100_batch, run_time_minutes, 1000, color="orange", label="Run Time")

    # Adding scatter points with large X markers on top of bars
    # ax.scatter(a100_batch, run_time_minutes, color='red', marker='x', s=100, label='Run Time Points')

    # Adding labels and title
    ax.set_xlabel("Batch Size")
    ax.set_ylabel("Run Time (minutes)")
    ax.set_title("Run Time by Batch Size gh200")

    # Display the plot
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_metrics_bar(a100_mae, a100_mse, a100_batch):
    # Number of bars
    n = len(a100_mae)

    # Create an array for the positions of the bars
    bar_width = 0.35
    index = np.arange(n)

    # Plotting the bars
    fig, ax = plt.subplots()
    bar1 = ax.bar(
        index,
        a100_mae,
        bar_width,
    )
    # bar2 = ax.bar(index + bar_width, a100_mse, bar_width, label='MSE')

    # Adding labels and title
    ax.set_xlabel("Batch Size")
    ax.set_ylabel("GPU Memory")
    ax.set_ylim(0, 90)
    ax.set_title("GPU Memory by Batch Size for a100")
    ax.set_xticks(index)
    ax.set_xticklabels(a100_batch)
    ax.legend()

    # Display the plot
    plt.show()

In [ ]:
plot_runtime_bar_chart(a100_batch, a100_runtime)

In [ ]:
plot_runtime_bar_chart(gh200_batch, gh200_runtime)

In [ ]:
plot_metrics_bar(a100_gpu, a100_mse, a100_batch)

In [ ]:
plot_metrics_bar(gh200_gpu, gh200_mse, gh200_batch)